In [2]:
!pip install bigframes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.6 MB/s eta 0:00:00


In [3]:
import bigframes.pandas as bpd

bpd.options.bigquery.ordering_mode="partial"
bpd.options.display.repr_mode="deferred"

In [ ]:
#Reading directly from google cloud
#df = bpd.read_gbq_table("bigquery-public-data.iowa_liquor_sales.sales")

#Let's read as a csv file into a bigframes pandas
url = "https://raw.githubusercontent.com/baursafi/GA_Projects/refs/heads/master/2_liquor_sales_iowa/data/Iowa_Liquor_sales_sample_10pct.csv"

# Read the CSV file directly from the GitHub URL into a BigQuery table
df = bpd.read_csv(url)

In [ ]:
#view data
df.peek() #head

In [ ]:
df.dtypes() #view data types

In [ ]:
df.describe("all").to_pandas() #view summary statistics

In [ ]:
volume_by_zip = df.groupby("zip_code").agg({"volume_sold_litres": "sum"})
volume_by_zip.plot.hist(bins=20)

In [ ]:
#bar plot
(
    volume_by_zip
    .sort_values("volume_sold_litres", ascending=False)
    .head(25) # head works because of ordering
    .to_pandas()
    .plot.bar(rot=80)
)

In [ ]:
#scatter plot
volume_by_pop = df.groupby("zip_code").agg({"volume_sold_litres": "sum", "population": "sum"})
(
    volume_by_pop[["volume_sold_litres", "population"]]
    .to_pandas()
    .plot.scatter(x="population", y="volume_sold_litres")
)

In [ ]:
# linear regression
from bigframes.ml.linear_model import LinearRegression
#from bigframes.ml.preprocessing import StandardScaler

feature_columns = volume_by_pop[["total_pop"]]
label_column = volume_by_pop[["volume_sold_litres"]]

# linear model
model = LinearRegression()
model.fit(feature_columns, label_column)


In [ ]:
# performance
model.score(feature_columns, label_column).to_pandas()

In [ ]:
df.category_name.nunique() #how many unique categories

In [ ]:
counts = (
    df.groupby("category_name")
    .agg({"volume_sold_litres": "sum"})
    .sort_values("volume_sold_litres", ascending=False)
    .to_pandas()
)

counts.head(25).plot.bar(rot=80)